In [ ]:
# !pip install opencv-python

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
curr_dir = os.getcwd()
dataset_type = ['training_set', 'testing_set']
image_type = ['image', 'labels']
ignore_dir = ['.DS_Store']
lesions_type = ['microaneurysms', 'haemorrhages', 'hard_exudates', 'soft_exudates']

In [ ]:
labels_dataset = os.path.join(curr_dir, 'dataset/labels')
images_dataset = os.path.join(curr_dir, 'dataset/images')

# Changing image labels to binary

In [ ]:
def create_binary_masks(infile, input_dir, output_dir = "", size = (4288, 2848)):
    outfile = os.path.splitext(infile)[0]
    extension = os.path.splitext(infile)[1]
    
    if (extension == ".jpg"):
        return
    
    if infile != outfile:
        try:
            # im = Image.open(os.path.join(label_dir, infile)) # slight modifications
            im = Image.open(os.path.join(input_dir, infile))
            gray = im.convert('L')
            bw = gray.point(lambda x: 0 if x < 50 else 255, '1')
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            bw.save(os.path.join(output_dir, outfile[:-3] + ".jpg"), "JPEG", quality = 100) # change this    
        except IOError:
            print("Cannot reduce image for", infile)

In [ ]:
for dataset in dataset_type:
    for lesions_folder in lesions_type:
        image_dir = os.path.join(labels_dataset, f'{dataset}/{lesions_folder}')
        output_dir = os.path.join(labels_dataset, f'{dataset}/{lesions_folder}_binary')
        for image in os.listdir(image_dir):
            create_binary_masks(image, image_dir, output_dir)

# Creating Images and Labels Patches

In [ ]:
def create_image_and_label_patch(file, image_dir, mask_dir, out_image_dir, out_label_dir):
#     for file in os.listdir(dir_mask):
    outfile = os.path.splitext(file)[0]
    extension = os.path.splitext(file)[1]

    im = Image.open(os.path.join(mask_dir, file))
    imd = Image.open(os.path.join(image_dir, file))
    
    if not os.path.exists(out_image_dir):
        os.makedirs(out_image_dir)
    if not os.path.exists(out_label_dir):
        os.makedirs(out_label_dir)

    patch_id = 0
    for i in range(10):
        for j in range(16):
            top_y = i*256
            if (i == 9):
                top_y = 2336
            top_x = j*256
            if(j == 15):
                top_x = 3776

            im_crop = im.crop((top_x, top_y, top_x + 512, top_y+512))
            imd_crop = imd.crop((top_x, top_y, top_x+512, top_y+512))

            im_crop.save(os.path.join(out_label_dir, outfile+"_p"+str(patch_id)+extension), "JPEG", quality = 100)
            imd_crop.save(os.path.join(out_image_dir, outfile+"_p"+str(patch_id)+extension), "JPEG", quality = 100)

            patch_id += 1

In [ ]:
for dataset in dataset_type:
    images_dir = os.path.join(images_dataset, f'{dataset}')
#     images_list = os.listdir(os.path.join(images_dataset, f'{dataset}'))
    for lesions_folder in lesions_type:
        label_images_dir = os.path.join(labels_dataset, f'{dataset}/{lesions_folder}_binary')
        
        # label_images_list = os.listdir(os.path.join(labels_dataset, f'{dataset}/{lesions_folder}_binary'))
        for image in os.listdir(label_images_dir):
            out_image_dir = os.path.join(images_dataset, f'{dataset}/{lesions_folder}_patch')
            out_label_dir = os.path.join(labels_dataset, f'{dataset}/{lesions_folder}_binary_patch')
            create_image_and_label_patch(image, images_dir, label_images_dir, out_image_dir, out_label_dir)

# Deleting Negative Patches

In [ ]:
def delete_negative_patches(image, images_patches_dir, label_images_patches_dir):
    mask_image = cv2.imread(os.path.join(label_images_patches_dir, image))
    if (mask_image.max() < 100):
        os.remove(os.path.join(label_images_patches_dir, image))
        os.remove(os.path.join(images_patches_dir, image))

In [ ]:
for dataset in dataset_type:
    images_dir = os.path.join(images_dataset, f'{dataset}')
    for lesions_folder in lesions_type:
        label_images_patches_dir = os.path.join(labels_dataset, f'{dataset}/{lesions_folder}_binary_patch')
        for image in os.listdir(label_images_patches_dir):
            images_patches_dir = os.path.join(images_dataset, f'{dataset}/{lesions_folder}_patch')
            delete_negative_patches(image, images_patches_dir, label_images_patches_dir)

# Creating Patches, Performing Operations and then Merging it

# (4288, 2848)

In [ ]:
# !pip install tensorflow
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.python.keras import models
import tensorflow as tf
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
# from tensorflow.python.keras import models
from tensorflow.keras.models import load_model
from tensorflow.python.keras import backend as K
import os
from scipy.special import expit
from pathlib import Path
from PIL import Image
import numpy as np


def dice_coeff(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss


def model():
    curr_dir = os.getcwd()
    # model_path = os.path.join(curr_dir, 'model/hard_exudates_segmentation_model.keras')
    # model_path = './model/hard_exudates_segmentation_model.keras'
    model = load_model('model/hard_exudates_segmentation_model.keras', custom_objects={'bce_dice_loss': bce_dice_loss,
                                                          'dice_loss': dice_loss})

    return model

def save_predicted_image(predicted_image, patch_id):
    predicted_image = np.squeeze(predicted_image, axis=0)
    predicted_image = np.squeeze(predicted_image)
    threshold = 0.9
    predicted_image = predicted_image > threshold
    predicted_mask = (predicted_image.astype('uint8'))*255
    output_image = Image.fromarray(predicted_mask)
    output_image.save(f'image{patch_id}.jpg', "JPEG", quality=100)
    return predicted_mask

def predict_hx(image, patch_id, image_WH = (256, 256)):
    myModel = model()
    # image = tf.image.decode_jpeg(image, channels=3)
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    image = tf.image.resize(image, (256, 256))/255.
    image = tf.expand_dims(image, axis=0)
    predicted_image = myModel.predict(image, steps=1)
    predicted_image = tf.image.resize(predicted_image, (512, 512))
    final_image = save_predicted_image(predicted_image, patch_id)
    return final_image

In [ ]:
curr_dir = os.getcwd()
sample_image = os.path.join(curr_dir, 'dataset/images/training_set')
filename = 'IDRiD_10.jpg'
outfile = os.path.splitext(filename)[0]
extension = os.path.splitext(filename)[1]
image = Image.open(os.path.join(sample_image, filename))
# image = np.asarray(image)

In [ ]:
predicted_image = np.zeros((2848, 4288), dtype=float)
patch_id = 0
for i in range(10): #10 6  
  for j in range(16): #16 9
    top_y = i * 256 #256 512
    if (i==9): #9 5
      top_y = 2336
    top_x = j * 256 #256 512
    if (j==15): #15 8
      top_x = 3776
    # image_crop = image[top_y:top_y+512, top_x:top_x+512]
    image_crop = image.crop((top_x, top_y, top_x + 512, top_y+512))
    predicted_crop = predict_hx(image_crop, patch_id)
    predicted_image[top_y:top_y+512, top_x:top_x+512] = np.maximum(predicted_image[top_y:top_y+512, top_x:top_x+512], predicted_crop)
    patch_id += 1

In [ ]:
predicted_save = Image.fromarray(predicted_image)
image = predicted_save.convert('RGB')
image.save('predicted_image.jpg', "JPEG", quality=100)

In [ ]:
print(f"TensorFlow version: {tf.__version__}")